In [1]:
from langchain_community.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

In [ ]:
persist_directory = "../chroma_db_experiment"

In [3]:
embedding_id = "mxbai-embed-large"

In [4]:
embedding = OllamaEmbeddings(model = embedding_id)

In [5]:
vector_store = Chroma(
    persist_directory = persist_directory,
    embedding_function=embedding
)

C:\Users\engma\AppData\Local\Temp\ipykernel_22940\3479936724.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
system_prompt = """
You are the lawyer and author of the contract agreement written in the organisation Indian Oil Corporation Limited.
Your job is to help the contractors and the officers of the organisation to do theiur job better by making them understand various clause agreements,
quote the necessary contract agreements if asked about the related contract agreements.
Only extract answers which are relevant to the question.
If no context is available, use the question as reference.
Context : {context}
"""

retriever = vector_store.as_retriever(search_kwargs = {'k' : 1})

prompt = ChatPromptTemplate.from_messages([("system", system_prompt),
                                           ("human", "{input}")])

from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_community.llms import Ollama

model_id_gemma = "gemma2:2b"
llm_url = "http://localhost:11434"

llm_gamma = Ollama(model=model_id_gemma, base_url=llm_url, temperature=0.2)

compressor = LLMChainExtractor.from_llm(llm_gamma)

compressor_retriever = ContextualCompressionRetriever(
    base_compressor = compressor, base_retriever = retriever
)

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

question_answer_chain_gamma = create_stuff_documents_chain(llm_gamma, prompt)
chain_gamma = create_retrieval_chain(compressor_retriever, question_answer_chain_gamma)

C:\Users\engma\AppData\Local\Temp\ipykernel_22940\4020431038.py:23: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm_gamma = Ollama(model=model_id_gemma, base_url=llm_url, temperature=0.2)


: 

In [ ]:
question = "How payment shall be done corresponding to measurement of the work done?"

response_sample_gamma = chain_gamma.invoke({"input" : question})
print("Gamma response: ", response_sample_gamma["answer"])
